---
title: "Build a CI/CD Pipeline using Github Actions and Terraform"
author: "Vahram Poghosyan"
date: "2024-12-20"
categories: ["Software Engineering", "Infrastructure", "Hosting"]
format:
  html:
    code-fold: false
    code-line-numbers: false
jupyter: python3
include-after-body:
  text: |
    <script type="application/javascript" src="../../javascript/light-dark.js"></script>
---

# Question 1

How can I write a pipeline that deploys AWS infrastructure using either Terraform or Ansible? Do I need other tools? If so, which?

# Answer 1

To create a pipeline that deploys AWS infrastructure using Terraform, you'll need to set up a Continuous Integration and Continuous Deployment (CI/CD) process. Here's how you can do it and the tools you'll need.

---

### **1. Use Terraform for Infrastructure Deployment**

**Terraform** is an open-source Infrastructure as Code (IaC) tool that allows you to define and provision infrastructure using code. It is well-suited for deploying AWS resources.

### **2. Set Up a Version Control System (VCS)**

Use a VCS like **Git** to manage your Terraform code. Host your repositories on platforms such as:

- **GitHub**
- **GitLab**
- **Bitbucket**

### **3. Choose a CI/CD Tool**

Select a CI/CD tool to automate your deployment pipeline. Popular options include:

- **Jenkins**
- **GitHub Actions**
- **GitLab CI/CD**
- **CircleCI**
- **AWS CodePipeline**

For seamless AWS integration, **AWS CodePipeline** is a good choice.

### **4. Configure AWS Credentials**

Securely manage AWS credentials required for deployment:

- Use **AWS Identity and Access Management (IAM)** roles and users.
- Store credentials securely using environment variables or the CI/CD tool's secret management features.
- **Never** hardcode credentials in your code.

### **5. Set Up Remote State Management**

Terraform uses state files to keep track of resource mappings. Store the state remotely:

- Use an **S3 bucket** with **DynamoDB** for state locking.
- Alternatively, use **Terraform Cloud** or **Terraform Enterprise**.

### **6. Build the Pipeline**

#### **a. Write Terraform Configuration**

Define your AWS infrastructure in `.tf` files:

```hcl
# provider.tf
provider "aws" {
  region = "us-east-1"
}

# main.tf
resource "aws_s3_bucket" "my_bucket" {
  bucket = "my-example-bucket"
  acl    = "private"
}
```

#### **b. Initialize Git Repository**

Initialize a Git repository and commit your Terraform code:

```bash
git init
git add .
git commit -m "Initial commit"
git push origin main
```

#### **c. Configure the CI/CD Pipeline**

##### **Option 1: Using GitHub Actions**

Create a workflow file `.github/workflows/deploy.yml`:

```yaml
name: Terraform Deploy

on:
  push:
    branches: [ main ]

jobs:
  terraform:
    runs-on: ubuntu-latest

    steps:
    - name: Checkout Code
      uses: actions/checkout@v3

    - name: Setup Terraform
      uses: hashicorp/setup-terraform@v2

    - name: Terraform Init
      run: terraform init

    - name: Terraform Apply
      run: terraform apply -auto-approve
      env:
        AWS_ACCESS_KEY_ID: ${{ secrets.AWS_ACCESS_KEY_ID }}
        AWS_SECRET_ACCESS_KEY: ${{ secrets.AWS_SECRET_ACCESS_KEY }}
```

- **Store AWS Credentials**: Add `AWS_ACCESS_KEY_ID` and `AWS_SECRET_ACCESS_KEY` to your repository's secrets.

##### **Option 2: Using AWS CodePipeline**

- **Create a Pipeline**: Set up AWS CodePipeline with stages for source, build, and deploy.
- **Use CodeBuild**: Configure AWS CodeBuild to run Terraform commands.

#### **d. Securely Manage State**

Configure Terraform to use remote state:

```hcl
# backend.tf
terraform {
  backend "s3" {
    bucket         = "my-terraform-state-bucket"
    key            = "state/terraform.tfstate"
    region         = "us-east-1"
    dynamodb_table = "terraform-state-lock"
    encrypt        = true
  }
}
```

#### **e. Test the Pipeline**

- **Commit and Push**: Your CI/CD tool will trigger the pipeline on new commits.
- **Review Outputs**: Monitor the pipeline for successful deployment.

### **7. Additional Tools**

- **Terraform Modules**: Reusable modules for organizing code.
- **Terragrunt**: Enhances Terraform with wrappers for state and configuration.
- **Linting and Formatting**:
  - Use `terraform fmt` for formatting.
  - Use `tflint` for linting your code.
- **Security Scanning**:
  - Integrate tools like **Checkov** or **TFSec** to scan for security issues.

### **8. Optional: If You Meant Ansible Instead of Angular**

**Ansible** is another automation tool that can manage infrastructure:

- Use Ansible playbooks to define AWS resources.
- Requires AWS modules and credentials setup.
- Can be integrated into a CI/CD pipeline similarly.

### **Important Notes**

- **Angular** is a web application framework and not used for infrastructure deployment. If you intended to refer to another tool (like Ansible), consider using that instead.
- **Testing**: Always test your infrastructure changes in a non-production environment first.
- **Monitoring**: Implement monitoring and logging for your infrastructure using AWS CloudWatch or similar services.

---

**Summary**

To deploy AWS infrastructure using Terraform:

1. **Write** your infrastructure code with Terraform.
2. **Use** a VCS to manage code changes.
3. **Set up** a CI/CD pipeline with a tool like GitHub Actions or AWS CodePipeline.
4. **Securely manage** AWS credentials and Terraform state.
5. **Automate** the deployment process through the pipeline.
6. **Monitor** and **maintain** your infrastructure post-deployment.

By integrating these tools and practices, you'll have an automated pipeline that helps you efficiently manage and deploy your AWS infrastructure.



# Question 2

"Store AWS Credentials: Add `AWS_ACCESS_KEY_ID` and `AWS_SECRET_ACCESS_KEY` to your repository's secrets."

Expand on this step. Also expand on "using an S3 bucket with DynamoDB for state locking."

# Answer 2

**Expanding on Storing AWS Credentials in Repository Secrets**

When setting up your CI/CD pipeline to deploy AWS infrastructure using Terraform, it's crucial to securely manage your AWS credentials. These credentials are necessary for Terraform to authenticate with AWS and provision resources on your behalf. Exposing these credentials in your code or configuration files poses significant security risks. Instead, store them securely using your repository's secret management features.

---

### **1. Storing AWS Credentials as Repository Secrets**

#### **a. Generate AWS Access Keys**

Before storing credentials, you need AWS access keys with appropriate permissions:

- **Create an IAM User:**
  1. Navigate to the [AWS IAM Console](https://console.aws.amazon.com/iam/).
  2. Click **Users** → **Add users**.
  3. Enter a user name (e.g., `terraform-ci-user`).
  4. Select **Programmatic access** for **Access type**.
  5. Click **Next: Permissions**.
  6. Assign permissions:
     - **Best Practice:** Attach policies granting least privilege necessary (e.g., custom policy or predefined policies like `AmazonS3FullAccess` and specific permissions for other services).
  7. Complete the steps to create the user.

- **Download Credentials:**
  - After creation, download the **Access Key ID** and **Secret Access Key**.
  - **Important:** Store these securely; you cannot retrieve the secret key again after this point.

#### **b. Store Credentials in Repository Secrets**

##### **GitHub Actions**

- **Navigate to Repository Settings:**
  - Go to your GitHub repository.
  - Click on the **Settings** tab.
  - In the left sidebar, select **Secrets and variables** → **Actions**.

- **Add New Secrets:**
  - Click **New repository secret**.
  - Add the **Access Key ID**:
    - **Name:** `AWS_ACCESS_KEY_ID`
    - **Value:** *Your AWS Access Key ID*
  - Add another secret for the **Secret Access Key**:
    - **Name:** `AWS_SECRET_ACCESS_KEY`
    - **Value:** *Your AWS Secret Access Key*

- **Reference Secrets in Workflow:**
  - In your `.github/workflows/deploy.yml`, reference the secrets:

    ```yaml
    env:
      AWS_ACCESS_KEY_ID: ${{ secrets.AWS_ACCESS_KEY_ID }}
      AWS_SECRET_ACCESS_KEY: ${{ secrets.AWS_SECRET_ACCESS_KEY }}
    ```

##### **GitLab CI/CD**

- **Navigate to Project Settings:**
  - Go to your GitLab project.
  - Click **Settings** → **CI/CD**.
  - Expand the **Variables** section.

- **Add Variables:**
  - Click **Add Variable**.
    - **Key:** `AWS_ACCESS_KEY_ID`
    - **Value:** *Your AWS Access Key ID*
    - **Flags:** Masked, Protected
  - Add another variable:
    - **Key:** `AWS_SECRET_ACCESS_KEY`
    - **Value:** *Your AWS Secret Access Key*
    - **Flags:** Masked, Protected

- **Reference Variables in `.gitlab-ci.yml`:**

  ```yaml
  variables:
    AWS_ACCESS_KEY_ID: $AWS_ACCESS_KEY_ID
    AWS_SECRET_ACCESS_KEY: $AWS_SECRET_ACCESS_KEY
  ```

#### **c. Security Best Practices**

- **Never Hardcode Credentials:**
  - Do not include credentials in your code or commit them to version control.
- **Use Least Privilege:**
  - Grant only the necessary permissions to the IAM user.
- **Rotate Credentials Regularly:**
  - Periodically update your access keys for better security.
- **Audit and Monitor Access:**
  - Use AWS CloudTrail and other monitoring tools to keep track of usage.

---

**Expanding on Using an S3 Bucket with DynamoDB for State Locking**

Terraform uses state files to keep track of the resources it manages. Storing the state remotely allows team collaboration and provides a central source of truth. Using an S3 bucket with DynamoDB for state locking ensures that only one operation can modify the state at a time, preventing conflicts.

---

### **2. Setting Up Remote State with S3 and DynamoDB**

#### **a. Create an S3 Bucket for Remote State**

1. **Navigate to S3 Console:**
   - Go to the [AWS S3 Console](https://s3.console.aws.amazon.com/s3/).

2. **Create a New Bucket:**
   - Click **Create bucket**.
   - **Bucket Name:** Must be globally unique (e.g., `my-terraform-state-bucket-1234`).
   - **Region:** Choose the AWS region closest to your resources.

3. **Configure Bucket Options:**
   - **Versioning:** Enable versioning to keep track of changes to the state file.
   - **Encryption:** Enable server-side encryption (SSE-S3 or SSE-KMS) for security.
   - **Block Public Access:** Ensure all public access is blocked.

4. **Review and Create:**
   - Review settings and create the bucket.

#### **b. Create a DynamoDB Table for State Locking**

1. **Navigate to DynamoDB Console:**
   - Go to the [AWS DynamoDB Console](https://console.aws.amazon.com/dynamodb/).

2. **Create a New Table:**
   - Click **Create table**.
   - **Table Name:** e.g., `terraform-lock-table`.
   - **Partition Key:** `LockID` (String).
   - **Settings:** Use default settings unless you have specific requirements.

3. **Create the Table:**
   - Complete the creation process.

#### **c. Configure Terraform Backend**

In your Terraform configuration, specify the backend as follows:

```hcl
# backend.tf

terraform {
  backend "s3" {
    bucket         = "my-terraform-state-bucket-1234"
    key            = "global/s3/terraform.tfstate"
    region         = "us-east-1"
    dynamodb_table = "terraform-lock-table"
    encrypt        = true
  }
}
```

- **bucket:** The name of your S3 bucket.
- **key:** The path within the bucket for the state file.
- **region:** AWS region for S3 and DynamoDB.
- **dynamodb_table:** Your DynamoDB table name.
- **encrypt:** Enables server-side encryption.

#### **d. Initialize Terraform with Backend**

- Run `terraform init` to initialize the backend configuration.

  ```bash
  terraform init
  ```

- Terraform will detect the backend settings and use the S3 bucket for state storage.

#### **e. IAM Permissions**

Ensure the IAM user or role used by Terraform has the necessary permissions:

- **S3 Permissions:**

  ```json
  {
    "Effect": "Allow",
    "Action": [
      "s3:PutObject",
      "s3:GetObject",
      "s3:DeleteObject",
      "s3:ListBucket"
    ],
    "Resource": [
      "arn:aws:s3:::my-terraform-state-bucket-1234",
      "arn:aws:s3:::my-terraform-state-bucket-1234/*"
    ]
  }
  ```

- **DynamoDB Permissions:**

  ```json
  {
    "Effect": "Allow",
    "Action": [
      "dynamodb:PutItem",
      "dynamodb:GetItem",
      "dynamodb:DeleteItem",
      "dynamodb:UpdateItem",
      "dynamodb:Scan",
      "dynamodb:Query"
    ],
    "Resource": "arn:aws:dynamodb:us-east-1:123456789012:table/terraform-lock-table"
  }
  ```

Replace `my-terraform-state-bucket-1234` with your bucket name and ensure the AWS account ID and region are correct.

#### **f. Benefits of Using S3 and DynamoDB**

- **Remote State Storage:**
  - Centralized state file accessible by your CI/CD pipeline and team members.
- **State Locking with DynamoDB:**
  - Prevents simultaneous operations that could corrupt the state.
- **Versioning and Encryption:**
  - State file history is preserved, and data is secured.

#### **g. Security Best Practices**

- **Restrict Access to State Files:**
  - Limit S3 bucket access to only necessary users and roles.
- **Encrypt Sensitive Data:**
  - Use `encrypt = true` in the backend configuration.
- **Monitor State Access:**
  - Use AWS CloudTrail to audit access to S3 and DynamoDB.

---

**Summary**

By securely storing your AWS credentials in repository secrets and configuring Terraform to use an S3 bucket with DynamoDB for remote state and locking, you enhance the security and reliability of your infrastructure deployment pipeline.

- **AWS Credentials:**
  - Use your CI/CD platform's secret management to store credentials securely.
  - Reference these secrets in your pipeline configuration.
  - Apply security best practices for credential management.

- **Terraform Remote State:**
  - Store the state file in an S3 bucket with versioning and encryption.
  - Use a DynamoDB table for state locking to prevent concurrent modifications.
  - Ensure IAM policies grant only the necessary permissions.

Feel free to ask if you need further details on any of these steps or assistance with configuring your pipeline.